# C/C++ Library with Python 

The advantage of Python is that it is **flexible and easy** to program. The time it takes to setup a new calulation is therefore short. 

But for certain types of calculations Python (and any other interpreted language) can be **very slow**.

Such calculations may be implemented in a compiled language such as C or Fortran.

In [ ]:
import seuif97
%timeit seuif97.pt2h(15,535)

In [ ]:
from iapws.iapws97 import IAPWS97
%timeit IAPWS97(P=16.10,T=535.10).h

## 1 ctypes - access the C library

**ctypes** is a foreign function library for Python. It provides C compatible data types, and allows calling functions in DLLs or shared libraries. It can be used to wrap these libraries in pure Python.

http://docs.python.org/3/library/ctypes.html

We need to load the library and set properties such as the functions return and argument types using the **ctypes** package :

1. **Loads the library** 

  **ctypes** exports the **cdll**, and on Windows **windll** objects, for loading dynamic link libraries.

   * **cdll.LoadLibrary(name)** : loads the library  which export functions using standard `__cdecl` calling convention
   * **windll.LoadLibrary(name)** : loads the library with `__stdcall` calling convention for the function 


2. **Specifying the required `argument` types:`argtypes`**  
  
  * It specify the required argument types of functions exported from DLLs by setting the argtypes attribute

3. **Return types: `restype`**

  * Return typescan be specified by setting the restype attribute of the function object.

4. **Function prototypes**
 
 * **CFUNCTYPE** 
 
   The CFUNCTYPE factory function creates types for callback functions using the normal cdecl calling convention
   
```python
ctypes.CFUNCTYPE(restype, *argtypes, use_errno=False, use_last_error=False)
``` 
 * **WINFUNCTYPE**
  
  Windows only: The returned function prototype creates functions that use the `__stdcall` calling convention
```python
ctypes.WINFUNCTYPE(restype, *argtypes, use_errno=False, use_last_error=False)
```

## 2 Call the Shared Library in from Python

### 2.1 C Library: libdemodll.dll 
 
#### C code 
calling convention

* `__cdecl` :

```c
int GetNumber(int *v)
```
 
* `__stdcall: 

```c
__declspec(dllexport) int __stdcall  GetNumber(int *v)
```

In [ ]:
%%file ./demo/src/demodll.c

int GetNumber(int *v); 

//__declspec(dllexport) int __stdcall  GetNumber(int *v); 

int GetNumber(int *v)
{
    int temp;
	temp=*v+1;
	*v=temp;
	return (*v)+1;
}


#### Build DLL 

In [ ]:
!gcc -c -O3 -Wall -fPIC -o ./demo/bin/demodll.o  ./demo/src//demodll.c
!gcc -shared -o ./demo/bin/libdemodll.dll  ./demo/bin/demodll.o

#### Call libdemodll.dll in C

In [ ]:
%%file ./demo/src/calldemodll.c
#include <stdio.h>
#include <stdlib.h>

int GetNumber(int *v);

int main()
{
    int v=10;
    int res=GetNumber(&v); // &v: the  address of the v 
    printf("%d %d",v,res);
	return 0;
}

In [ ]:
!gcc  -o ./demo/bin/calldemodll  ./demo/src/calldemodll.c -L./demo/bin/ -llibdemodll

In [ ]:
!.\demo\bin\calldemodll

### 2.3  C compatible data types

**Fundamental data types**

ctypes defines a number of primitive C compatible data types:

https://docs.python.org/3/library/ctypes.html#fundamental-data-types



<table class="docutils" border="1">
<colgroup>
<col width="24%">
<col width="46%">
<col width="30%">
</colgroup>
<thead valign="bottom">
<tr class="row-odd"><th class="head">ctypes type</th>
<th class="head">C type</th>
<th class="head">Python type</th>
</tr>
</thead>
<tbody valign="top">
<tr class="row-even"><td><a title="ctypes.c_bool" class="reference internal" href="#ctypes.c_bool"><code class="xref py py-class docutils literal notranslate"><span class="pre">c_bool</span></code></a></td>
<td><code class="xref c c-type docutils literal notranslate"><span class="pre">_Bool</span></code></td>
<td>bool (1)</td>
</tr>
<tr class="row-odd"><td><a title="ctypes.c_char" class="reference internal" href="#ctypes.c_char"><code class="xref py py-class docutils literal notranslate"><span class="pre">c_char</span></code></a></td>
<td><code class="xref c c-type docutils literal notranslate"><span class="pre">char</span></code></td>
<td>1-character bytes object</td>
</tr>
<tr class="row-even"><td><a title="ctypes.c_wchar" class="reference internal" href="#ctypes.c_wchar"><code class="xref py py-class docutils literal notranslate"><span class="pre">c_wchar</span></code></a></td>
<td><code class="xref c c-type docutils literal notranslate"><span class="pre">wchar_t</span></code></td>
<td>1-character string</td>
</tr>
<tr class="row-odd"><td><a title="ctypes.c_byte" class="reference internal" href="#ctypes.c_byte"><code class="xref py py-class docutils literal notranslate"><span class="pre">c_byte</span></code></a></td>
<td><code class="xref c c-type docutils literal notranslate"><span class="pre">char</span></code></td>
<td>int</td>
</tr>
<tr class="row-even"><td><a title="ctypes.c_ubyte" class="reference internal" href="#ctypes.c_ubyte"><code class="xref py py-class docutils literal notranslate"><span class="pre">c_ubyte</span></code></a></td>
<td><code class="xref c c-type docutils literal notranslate"><span class="pre">unsigned</span> <span class="pre">char</span></code></td>
<td>int</td>
</tr>
<tr class="row-odd"><td><a title="ctypes.c_short" class="reference internal" href="#ctypes.c_short"><code class="xref py py-class docutils literal notranslate"><span class="pre">c_short</span></code></a></td>
<td><code class="xref c c-type docutils literal notranslate"><span class="pre">short</span></code></td>
<td>int</td>
</tr>
<tr class="row-even"><td><a title="ctypes.c_ushort" class="reference internal" href="#ctypes.c_ushort"><code class="xref py py-class docutils literal notranslate"><span class="pre">c_ushort</span></code></a></td>
<td><code class="xref c c-type docutils literal notranslate"><span class="pre">unsigned</span> <span class="pre">short</span></code></td>
<td>int</td>
</tr>
<tr class="row-odd"><td><a title="ctypes.c_int" class="reference internal" href="#ctypes.c_int"><code class="xref py py-class docutils literal notranslate"><span class="pre">c_int</span></code></a></td>
<td><code class="xref c c-type docutils literal notranslate"><span class="pre">int</span></code></td>
<td>int</td>
</tr>
<tr class="row-even"><td><a title="ctypes.c_uint" class="reference internal" href="#ctypes.c_uint"><code class="xref py py-class docutils literal notranslate"><span class="pre">c_uint</span></code></a></td>
<td><code class="xref c c-type docutils literal notranslate"><span class="pre">unsigned</span> <span class="pre">int</span></code></td>
<td>int</td>
</tr>
<tr class="row-odd"><td><a title="ctypes.c_long" class="reference internal" href="#ctypes.c_long"><code class="xref py py-class docutils literal notranslate"><span class="pre">c_long</span></code></a></td>
<td><code class="xref c c-type docutils literal notranslate"><span class="pre">long</span></code></td>
<td>int</td>
</tr>
<tr class="row-even"><td><a title="ctypes.c_ulong" class="reference internal" href="#ctypes.c_ulong"><code class="xref py py-class docutils literal notranslate"><span class="pre">c_ulong</span></code></a></td>
<td><code class="xref c c-type docutils literal notranslate"><span class="pre">unsigned</span> <span class="pre">long</span></code></td>
<td>int</td>
</tr>
<tr class="row-odd"><td><a title="ctypes.c_longlong" class="reference internal" href="#ctypes.c_longlong"><code class="xref py py-class docutils literal notranslate"><span class="pre">c_longlong</span></code></a></td>
<td><code class="xref c c-type docutils literal notranslate"><span class="pre">__int64</span></code> or <code class="xref c c-type docutils literal notranslate"><span class="pre">long</span> <span class="pre">long</span></code></td>
<td>int</td>
</tr>
<tr class="row-even"><td><a title="ctypes.c_ulonglong" class="reference internal" href="#ctypes.c_ulonglong"><code class="xref py py-class docutils literal notranslate"><span class="pre">c_ulonglong</span></code></a></td>
<td><code class="xref c c-type docutils literal notranslate"><span class="pre">unsigned</span> <span class="pre">__int64</span></code> or
<code class="xref c c-type docutils literal notranslate"><span class="pre">unsigned</span> <span class="pre">long</span> <span class="pre">long</span></code></td>
<td>int</td>
</tr>
<tr class="row-odd"><td><a title="ctypes.c_size_t" class="reference internal" href="#ctypes.c_size_t"><code class="xref py py-class docutils literal notranslate"><span class="pre">c_size_t</span></code></a></td>
<td><code class="xref c c-type docutils literal notranslate"><span class="pre">size_t</span></code></td>
<td>int</td>
</tr>
<tr class="row-even"><td><a title="ctypes.c_ssize_t" class="reference internal" href="#ctypes.c_ssize_t"><code class="xref py py-class docutils literal notranslate"><span class="pre">c_ssize_t</span></code></a></td>
<td><code class="xref c c-type docutils literal notranslate"><span class="pre">ssize_t</span></code> or
<code class="xref c c-type docutils literal notranslate"><span class="pre">Py_ssize_t</span></code></td>
<td>int</td>
</tr>
<tr class="row-odd"><td><a title="ctypes.c_float" class="reference internal" href="#ctypes.c_float"><code class="xref py py-class docutils literal notranslate"><span class="pre">c_float</span></code></a></td>
<td><code class="xref c c-type docutils literal notranslate"><span class="pre">float</span></code></td>
<td>float</td>
</tr>
<tr class="row-even"><td><a title="ctypes.c_double" class="reference internal" href="#ctypes.c_double"><code class="xref py py-class docutils literal notranslate"><span class="pre">c_double</span></code></a></td>
<td><code class="xref c c-type docutils literal notranslate"><span class="pre">double</span></code></td>
<td>float</td>
</tr>
<tr class="row-odd"><td><a title="ctypes.c_longdouble" class="reference internal" href="#ctypes.c_longdouble"><code class="xref py py-class docutils literal notranslate"><span class="pre">c_longdouble</span></code></a></td>
<td><code class="xref c c-type docutils literal notranslate"><span class="pre">long</span> <span class="pre">double</span></code></td>
<td>float</td>
</tr>
<tr class="row-even"><td><a title="ctypes.c_char_p" class="reference internal" href="#ctypes.c_char_p"><code class="xref py py-class docutils literal notranslate"><span class="pre">c_char_p</span></code></a></td>
<td><code class="xref c c-type docutils literal notranslate"><span class="pre">char</span> <span class="pre">*</span></code> (NUL terminated)</td>
<td>bytes object or <code class="docutils literal notranslate"><span class="pre">None</span></code></td>
</tr>
<tr class="row-odd"><td><a title="ctypes.c_wchar_p" class="reference internal" href="#ctypes.c_wchar_p"><code class="xref py py-class docutils literal notranslate"><span class="pre">c_wchar_p</span></code></a></td>
<td><code class="xref c c-type docutils literal notranslate"><span class="pre">wchar_t</span> <span class="pre">*</span></code> (NUL terminated)</td>
<td>string or <code class="docutils literal notranslate"><span class="pre">None</span></code></td>
</tr>
<tr class="row-even"><td><a title="ctypes.c_void_p" class="reference internal" href="#ctypes.c_void_p"><code class="xref py py-class docutils literal notranslate"><span class="pre">c_void_p</span></code></a></td>
<td><code class="xref c c-type docutils literal notranslate"><span class="pre">void</span> <span class="pre">*</span></code></td>
<td>int or <code class="docutils literal notranslate"><span class="pre">None</span></code></td>
</tr>
</tbody>
</table>
 
All these types can be created by calling them with an optional initializer of the correct type and value:

In [ ]:
from ctypes import *
v=c_int(10)
print(v)
print(v.value)
print(pointer(v))

###  2.4 Using C compatible data types to Call DLL

`cdll` loads libraries which export functions using the `__cdecl` calling convention 

In [ ]:
from ctypes import *

flib = cdll.LoadLibrary("./demo/bin/libdemodll.dll")

# flib = windll.LoadLibrary("./demo/bin/libdemodll.dll")

## 1 argtypes,restype
flib.GetNumber.restype=c_int
flib.GetNumber.argtypes= [POINTER(c_int)]
v=c_int(10)
pi = pointer(v)
print(flib.GetNumber(pi))

## 2 Function prototypes
prototype = WINFUNCTYPE(c_int,POINTER(c_int))
f = prototype(("GetNumber", flib),)
print(f(pi))

## 3 Type conversions
f=flib.GetNumber
v=c_int(10)
print(f(byref(v)))

### 2.5  Wrapper the Shared Library to the Python API 

SEUIF97.dll

* `__stdcall` calling convention: **windll, WINFUNCTYPE**


In [ ]:
from ctypes import *

flib = windll.LoadLibrary('libseuif97.dll')
prototype = WINFUNCTYPE(c_double, c_double, c_double, c_int)

# ---(p,t) ----------------
def pt(p, t, pid):
    f = prototype(("seupt", flib),)
    result = f(p, t, pid)
    return result


def pt2h(p, t):
    f = prototype(("seupt", flib),)
    result = f(p, t, 4)
    return result

In [ ]:
h=pt(16,535,4)
h

## 3 The Example:  Structure , Arrays 

The examples show that how to use **C compatible data type** to call functions in DLLs

* Structure

* Arrays(one/two-dimensional)

### 3.1 The Functions in C

In [ ]:
%%file ./demo/src/democtypes.c

// 1 Struct
typedef struct
{
    int nNo;
    float fValue;
} SimpleStruct;

__declspec(dllexport)  void __stdcall  TestSimpleStruct(SimpleStruct v,SimpleStruct *res)
{
  res->fValue= v.fValue+2;
  res->nNo=v.nNo+2;
  // return res->nNo;
}

// 2 one-dimensional array :
//    return: *narray: ith ,item =i
__declspec(dllexport)  void  __stdcall  TestArray1(double *narray,int size)
{
  for(int i=0; i<size; i++)
  {
     narray[i]=i;  
  }
}

// 3  two-dimensional array : double **ptr
//       return:  2*item
__declspec(dllexport)  void  __stdcall  TestArray22(double **ptr,int row, int col)
{
    int i, j;
    for(i=0; i<row; i++)
    {
        for(j=0; j<col; j++)
        {
            ptr[i][j]=2*ptr[i][j];
        }
	} 
}

In [ ]:
!gcc -c -O3 -Wall -fPIC -o ./demo/obj/democtypes.o  ./demo/src/democtypes.c
!gcc -shared -o ./demo/bin/libdemoctypes.dll ./demo/obj/democtypes.o
!del .\demo\obj\democtypes.o

### 3.2  using ctypes to call DLL : Structures

[Structures](https://docs.python.org/3/library/ctypes.html#structures-and-unions) must derive from the `Structure` base classes which are defined in the `ctypes` module.

Each subclass must define a `_fields_` attribute.

`_fields_` must be `a list of 2-tuples`, containing a field `name` and a field `type`: 

```python
_fields_=[(a field name,a field `type`)]
```

In [ ]:
from ctypes import *

flib=windll.LoadLibrary("./demo/bin/libdemoctypes.dll")

class SimpStruct(Structure):
    _fields_ = [("nNo", c_int),
                ("fValue", c_float)]

#  TestSimpleStruct
f1=flib.TestSimpleStruct
v = SimpStruct()
v.nNo = 16
v.fValue = 3.14

result = SimpStruct()

f1(v,byref(result))

print("v.fValue =" ,v.fValue)
print("result.fValue =",result.fValue)

### 3.3  using ctypes to call DLL :one/two-dimensional Array

[Arrays](https://docs.python.org/3/library/ctypes.html#arrays) are sequences, containing a fixed number of instances of the same type.

**create array types**

The recommended way to `create array types` is by

* multiplying a data type with a positive integer:

**one-dimensional array**

In [ ]:
# c_double*10
narray=(c_double*10)()  
print(type(narray))
print(narray[2])
print(list(narray))

In [ ]:
# c_double*len(list1)
list1=[1,2,3]
narray=(c_double*len(list1))(*list1)
print(narray)
print(narray[1])
print(list(narray))

**two-dimensional array**

In [ ]:
list2=[[1,2,3],[4,5,6]]
row=len(list2)
col=len(list2[0])
indata = (POINTER(c_double) * row)()
for i in range(row):
    indata[i] =(c_double*col)(*list2[i])
    print([indata[i][j] for j in range(col)])

In [ ]:
from ctypes import *

flib=windll.LoadLibrary("./demo/bin/libdemoctypes.dll")

#  one-dimensional array
# create one-dimensional array
size=10
narray=(c_double*size)()  
# using the name of one-dimensional array,non byref
# ith ,item =i
f=flib.TestArray1
f(narray,size)
print('one-dimensional array:8th',narray[8])

# two-dimensional array
# create two-dimensional array: **ptr
print("two-dimensional array,**ptr")
row=5
col=6

indata = (POINTER(c_double) * row)()
for i in range(row):
    # Allocate arrays of double
    indata[i] = (c_double * col)()
    for j in range(col):
        indata[i][j] = i+j
    ldata = [indata[i][j] for j in range(col)]
    print(ldata)

# using the name of two-dimensional array,byref
f=flib.TestArray22
# 2*item
f(byref(indata),row,col)
print("two-dimensional array,byref")
for i in range(row):
    print([indata[i][j] for j in range(col)])  


## 4 Example: Polynomial Regression Library 

**Polynomial Regression** 

https://en.wikipedia.org/wiki/Polynomial_regression
    
### 4.1   polynomial regression：Matrix form and calculation of estimates

**The polynomial regression model**

$${\displaystyle y_{i}\,=\,\beta _{0}+\beta _{1}x_{i}+\beta _{2}x_{i}^{2}+\cdots +\beta _{m}x_{i}^{m}+\varepsilon _{i}\ (i=1,2,\dots ,n)}$$

can be expressed in matrix form in terms of a design matrix ${\displaystyle \mathbf {X}}$ , a response vector ${\displaystyle ${\vec {y}}}$, a parameter vector ${\displaystyle {\vec {\beta }}}$, and a vector ${\displaystyle {\vec {\varepsilon }}}$ of random errors. 

The i-th row of ${\displaystyle \mathbf {X} }$  and ${\displaystyle {\vec {y}}}$ will contain the x and y value for the $i$-th data sample. Then the model can be written as a system of linear equations:

$${\displaystyle {\begin{bmatrix}y_{1}\\y_{2}\\y_{3}\\\vdots \\y_{n}\end{bmatrix}}={\begin{bmatrix}1&x_{1}&x_{1}^{2}&\dots &x_{1}^{m}\\1&x_{2}&x_{2}^{2}&\dots &x_{2}^{m}\\1&x_{3}&x_{3}^{2}&\dots &x_{3}^{m}\\\vdots &\vdots &\vdots &\ddots &\vdots \\1&x_{n}&x_{n}^{2}&\dots &x_{n}^{m}\end{bmatrix}}{\begin{bmatrix}\beta _{0}\\\beta _{1}\\\beta _{2}\\\vdots \\\beta _{m}\end{bmatrix}}+{\begin{bmatrix}\varepsilon _{1}\\\varepsilon _{2}\\\varepsilon _{3}\\\vdots \\\varepsilon _{n}\end{bmatrix}},}$$

which when using pure matrix notation is written as
$${\displaystyle {\vec {y}}=\mathbf {X} {\vec {\beta }}+{\vec {\varepsilon }}.\,}$$

The vector of estimated polynomial regression coefficients (using [ordinary least squares estimation](https://en.wikipedia.org/wiki/Estimation)) is

$${\displaystyle {\widehat {\vec {\beta }}}=(\mathbf {X} ^{\mathsf {T}}\mathbf {X} )^{-1}\;\mathbf {X} ^{\mathsf {T}}{\vec {y}},\,}$$

assuming $m < n$ which is required for the matrix to be invertible; then since ${\displaystyle \mathbf {X} }$  is a [Vandermonde matrix](https://en.wikipedia.org/wiki/Vandermonde_matrix), the invertibility condition is guaranteed to hold if all the ${\displaystyle x_{i}}$ values are distinct. This is the unique **least-squares solution**.

#### 4.1.1 C code

In [ ]:
%%file ./demo/src/PolynomialFit.c

#include <math.h>

void PolynomialFit(double x[], double y[], int size, int n, double a[])
{
    int i, j, k;
    double X[2 * n + 1]; //Array that will store the values of sigma(xi),sigma(xi^2),....sigma(xi^2n)
    for (i = 0; i < 2 * n + 1; i++)
    {
        X[i] = 0;
        for (j = 0; j < size; j++)
            X[i] = X[i] + pow(x[j], i); //consecutive positions of the array will store N,sigma(xi),sigma(xi^2),....sigma(xi^2n)
    }
    double B[n + 1][n + 2]; // B is the Normal matrix(augmented) that will store the equations,'a'is for value of the final coefficients
    for (i = 0; i <= n; i++)
        for (j = 0; j <= n; j++)
            B[i][j] = X[i + j]; // Build the Normal matrix by storing the corresponding coefficients at the right positions except the last column of the matrix
    double Y[n + 1];            // Array to store the values of sigma(yi),sigma(xi*yi),sigma(xi^2*yi)...sigma(xi^n*yi)
    for (i = 0; i < n + 1; i++)
    {
        Y[i] = 0;
        for (j = 0; j < size; j++)
            Y[i] = Y[i] + pow(x[j], i) * y[j]; //consecutive positions will store sigma(yi),sigma(xi*yi),sigma(xi^2*yi)...sigma(xi^n*yi)
    }
    for (i = 0; i <= n; i++)
        B[i][n + 1] = Y[i]; // load the values of Y as the last column of B(Normal Matrix but augmented)
    n = n + 1;              // n is made n+1 because the Gaussian Elimination part below was for n equations, but here n is the degree of polynomial and for n degree we get n+1 equations
    for (i = 0; i < n; i++) // From now Gaussian Elimination starts(can be ignored) to solve the set of linear equations (Pivotisation)
        for (k = i + 1; k < n; k++)
            if (B[i][i] < B[k][i])
                for (j = 0; j <= n; j++)
                {
                    double temp = B[i][j];
                    B[i][j] = B[k][j];
                    B[k][j] = temp;
                }

    for (i = 0; i < n - 1; i++) // loop to perform the gauss elimination
        for (k = i + 1; k < n; k++)
        {
            double t = B[k][i] / B[i][i];
            for (j = 0; j <= n; j++)
                B[k][j] = B[k][j] - t * B[i][j]; // make the elements below the pivot elements equal to zero or elimnate the variables
        }
    for (i = n - 1; i >= 0; i--) // back-substitution
    {                            // x is an array whose values correspond to the values of x,y,z..
        a[i] = B[i][n];          //make the variable to be calculated equal to the rhs of the last equation
        for (j = 0; j < n; j++)
            if (j != i) //then subtract all the lhs values except the coefficient of the variable whose value                                   is being calculated
                a[i] = a[i] - B[i][j] * a[j];
        a[i] = a[i] / B[i][i]; // now finally divide the rhs by the coefficient of the variable to be calculated
    }
};


In [ ]:
!gcc -o  ./demo/bin/DemoPolynomialFit ./demo/src/DemoPolynomialFit.c ./demo/src/PolynomialFit.c

In [ ]:
!.\demo\bin\DemoPolynomialFit

#### 4.1.2 Build the libPolynomialFit.dll

In [ ]:
!gcc -c -O3 -Wall -fPIC -o ./demo/obj/PolynomialFit.o  ./demo/src/PolynomialFit.c
!gcc -shared -o ./demo/bin/libPolynomialFit.dll ./demo/obj/PolynomialFit.o
!del .\demo\obj\PolynomialFit.o

### 4.2 Call Library in C

In [ ]:
%%file ./demo/data/springData.csv
Distance(m),Mass(kg)
0.0865,0.1
0.1015,0.15
0.1106,0.2
0.1279,0.25
0.1892,0.3
0.2695,0.35
0.2888,0.4
0.2425,0.45
0.3465,0.5
0.3225,0.55
0.3764,0.6
0.4263,0.65
0.4562,0.7
0.4502,0.75
0.4499,0.8
0.4534,0.85
0.4416,0.9
0.4304,0.95
0.437,1.0

#####  `strtok_r()`
C provides the functions `strtok_r()` for splitting a string by some delimiter

```c
char *strtok_r(char *str, const char *delim, char **saveptr);
```

*  The third argument `saveptr` is a pointer to a `char *` variable that is used internally by `strtok_r()` in  order to maintain context between `successive calls`

* On the `first` call to `strtok_r()`, `str` should point to `the string to be parsed`, and the value of `saveptr` is ignored.

* In `subsequent calls`, `str` should be `NULL`, and `saveptr` should be unchanged since the previous call.


In [1]:
%%file ./demo/src/DemoPolynomialFit.c
/*
   The Demo of Simple PolynomialFit 
*/

#include <stdio.h>
#include <string.h>
#include <stdlib.h>

void PolynomialFit(double x[], double y[], int size, int n, double a[]);

int main()
{

   int size = 19;
   double distances[size];
   double forces[size];

   FILE *fp = fopen("./demo/data/springData.csv", "r");
   if (fp == NULL)
   {
      fprintf(stderr, "failed to open file for reading\n");
      return 1;
   }

   char line[1024];
   fgets(line, sizeof(line), fp);
   int i = 0;
   while (fgets(line, sizeof(line), fp))
   {
      char *save_ptr;
      // The first call to strtok_r(), str point to the string to be parsed" line
      char *d = strtok_r(line, ",", &save_ptr);
      // In subsequent calls, str is NULL, and saveptr is unchanged since the previous call.
      char *m = strtok_r(NULL, ",", &save_ptr);
      distances[i] = atof(d);
      forces[i] = atof(m) * 9.81;
      i++;
   };
   fclose(fp);

   int n = 1; // n is the degree of Polynomial
   double a[n + 1];
   PolynomialFit(forces, distances, size - 6, n, a);
   printf(" PolynomialFit: k = %.6f", 1 / a[1]);
   return 0;
}

Overwriting ./demo/src/DemoPolynomialFit.c


In [2]:
!gcc -o  ./demo/bin/DemoPolynomialFit ./demo/src/DemoPolynomialFit.c -L./demo/bin/ -lPolynomialFit

In [3]:
!.\demo\bin\DemoPolynomialFit

 PolynomialFit: k = 15.453365


### 4.3 Call in Python

[UNDERSTANDING_EXPERIMENTAL_DATA:The Behavior of Springs](Unit2-3-UNDERSTANDING_EXPERIMENTAL_DATA.ipynb)

In [ ]:
import numpy as np
from ctypes import *


def getData(fileName):
    dataFile = open(fileName, 'r')
    distances = []
    forces = []
    discardHeader = dataFile.readline() # Distance(m),Mass(kg)
    for line in dataFile:
        d, m = line.split(',')
        distances.append(float(d))
        forces.append(float(m)*9.81) # m*9.81 -> force
    dataFile.close()
    return (forces, distances)

inputFile='./demo/data/springData.csv'
forces, distances = getData(inputFile)

forces=forces[:-6]
distances=distances[:-6]

# call the PolynomialFit Library by ctypes
flib=cdll.LoadLibrary("./demo/bin/libPolynomialFit.dll")
size=len(forces)
y=(c_double*size)(*distances)
x=(c_double*size)(*forces)
n=1
c=(c_double*(n+1))()
flib.PolynomialFit(byref(x), byref(y), size, n, byref(c));

print("Linear Fit:")
print("\tPolynomialFit:k =",1/c[1]);
# np.polyfit
a,b= np.polyfit(forces,distances, 1)    
print("\tnp.polyfit: k =",1/a);


## Reference

Python ctypes http://docs.python.org/3/library/ctypes.html

